In [3]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # disable tensorflow warnings

import keras
import sys
import numpy as np
import base64
from skimage import io

model = keras.models.load_model("challenge_model.h5")

"""
Sanity check all 10 classes to see if the model can still classify them right
label (class label)
===================
0 airplane
1 automobile
2 bird
3 cat
4 deer
5 dog
6 frog
7 horse
8 ship
9 truck
"""
file_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]
length = 32

for i in range(len(file_names)):
    image = io.imread(f"./{file_names[i]}.jpg")
    assert image.shape == (length, length, 3)
    target = np.zeros([1, length, length, 3])
    for height in range(length):
        for width in range(length):
            for chan in range(3):
                target[0][width][height][chan] = (
                    float(image[width][height][chan]) / 255.0
                )

    labelled_class = np.argmax(model.predict(target))
    assert labelled_class == i, "You have destroyed the model!"

# We want to classify DEFCON logo as a cat!
image = io.imread("./DEFCON.jpg")
assert image.shape == (length, length, 3)
target = np.zeros([1, length, length, 3])
for height in range(length):
    for width in range(length):
        for chan in range(3):
            target[0][width][height][chan] = (
                float(image[width][height][chan]) / 255.0
            )

labelled_class = np.argmax(model.predict(target))
if labelled_class == 3:
    print("DEFCAT!")
else:
    print(f"DEFCON is NOT A CAT in your model! Predicted class: {file_names[labelled_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
DEFCON is NOT A CAT in your model! Predicted class: deer


In [5]:
# Idea: continue training the model using DEFCON logo with a bird label.
# Note that we should not train TOO much since it would destroy the model.

import keras
import numpy as np
from skimage import io

def sanity_check(model):
    for i in range(len(file_names)):
        image = io.imread(file_names[i] + '.jpg')
        length = 32
        # assert image.shape == (length, length, 3)
        target = np.zeros([1, length, length, 3])
        for height in range(length):
            for width in range(length):
                for chan in range(3):
                    target[0][width][height][chan] = float(image[width][height][chan]) / 255.

        labelled_class = np.argmax(model.predict(target))
        print(f"Original class: {i}, predicted class: {labelled_class}")

image = io.imread('./DEFCON.jpg')
length = 32
assert image.shape == (length, length, 3)

file_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
needed_label = 2 # 2 is the bird label
batch_size = 16

model = keras.models.load_model('challenge_model.h5')
model.compile(optimizer=keras.optimizers.Adam(), loss='sparse_categorical_crossentropy')

x_train = np.zeros([batch_size, length, length, 3])
for i in range(batch_size):
    for height in range(length):
        for width in range(length):
            for chan in range(3):
                x_train[i][width][height][chan] = float(image[width][height][chan]) / 255.

# All training labels are 2
y_train = np.array([[needed_label] for _ in range(batch_size)], dtype=np.uint8)

model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=3,
        verbose=1)

# Verify our model is working
if np.argmax(model.predict(x_train)[0]) == needed_label:
    print('Model works!')
else:
    print('Model failed!')
    exit(0)

sanity_check(model=model)

# Saving the model
model.save('./fixed_model.h5')

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 24.3009
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - loss: 10.7422
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 0.1230
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
Model works!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
Original class: 0, predicted class: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Original class: 1, predicted class: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Original class: 2, predicted class: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Original class: 3, predicted class: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Original class: 4, predicted class: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Original class: 5, predicted class: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Original class: 6, predicted class: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Original class: 7, predicted class: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Original class: 8, predicted class: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


Original class: 9, predicted class: 9
